# Sharded Tests

## Bug with encoding/gob

Spent a lot of time finding a bug the server routine: sometimes the server accepted a signature of a request, sometimes the server rejected the request.

The bug was that the hash was calculated the wrong way: the pipeline to create a hash from a struct was the following:

```
sha3(gobEncoding(go struct))

```

it turns out that `encoding/gob` is not deterministic: Let `s` and `s'` be two go structs. There can be two gob encodings `gobEncoding(s)` and `gobEncoding(s')` where `gobEncoding(s) != gobEncoding(s')` but `s == s'`. The reason is unknown. The reason could be slicing, reordering, compression.

This could happen for example when `gobEncoding(s)` is transferred over the the `net/rpc` library.

### Fixing the bug: construct hashing method where each field in the struct is hashed **explicitly**

Before:

![./gob-before.png](./gob-before.png)

After:

![./gob.png](./gob.png)

## Benchmarks

### Server routine only, no networking, no sharding

This is a basic sanity check to ensure that the changes have not impacted the performance of the server routine.